# CART Amélioré

In [121]:
import numpy as np
import pandas as pd

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

In [25]:
prices = pd.read_csv("../data/price_availability.csv", sep=";")
listings = pd.read_csv("../clean/listings_final.csv", sep=";")
listings = listings.drop(589)  # Cet individu n'a pas de prix !
print("Chargement des données effectué.")

Chargement des données effectué.


In [14]:
uniques = np.unique(prices["listing_id"]) 
intersects = np.intersect1d(uniques, np.unique(listings["listing_id"]))
listings_prices = prices[prices["listing_id"].isin(intersects)]

In [15]:
dates =  pd.to_datetime(listings_prices["day"], format='%Y-%m-%d')
week_prices = listings_prices.assign(date=dates).set_index(dates).groupby(['listing_id', pd.Grouper(key='date', freq='7D')]).mean().reset_index().loc[:, ["listing_id", "date", "local_price"]]
week_prices.head()

,listing_id,date,local_price
0,56093,2018-08-27,170.0
1,56093,2018-09-03,170.0
2,56093,2018-09-10,170.0
3,56093,2018-09-17,170.0
4,56093,2018-09-24,170.0


In [66]:
week_prices.shape

(18981, 3)

In [67]:
week_prices.head()

,listing_id,date,local_price
0,56093,2018-08-27,170.0
1,56093,2018-09-03,170.0
2,56093,2018-09-10,170.0
3,56093,2018-09-17,170.0
4,56093,2018-09-24,170.0


In [87]:
def populate(row):
    return row.append(listings[listings["listing_id"] == row["listing_id"]].iloc[0,2:])
    
X = week_prices.apply(populate, axis=1)

In [89]:
X.shape

(18981, 20)

In [128]:
label_encoder = LabelEncoder()
X["label_dates"] =  label_encoder.fit_transform(X["date"])

In [129]:
X.shape

(18981, 21)

In [137]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X.loc[:, ["label_dates", "latitude", "longitude", "person_capacity", "bedrooms", "bathrooms"]],
    X.loc[:, "local_price"],
    test_size=0.2)

In [138]:
my_tree = DecisionTreeRegressor(max_depth=8)
my_tree.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=8, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [139]:
print('Variance score: %.4f' % r2_score(Y_test, my_tree.predict(X_test)))

Variance score: 0.7333


In [133]:
listings.columns

Index(['Unnamed: 0', 'listing_id', 'name', 'type', 'city', 'neighborhood',
       'latitude', 'longitude', 'person_capacity', 'beds', 'bedrooms',
       'bathrooms', 'is_rebookable', 'is_new_listing', 'is_fully_refundable',
       'is_host_highly_rated', 'is_business_travel_ready',
       'pricing_weekly_factor', 'pricing_monthly_factor'],
      dtype='object')